In [941]:
# This is the generic code for health impact assessment (HIA) and burden of disease (BoD) calculations
# Forked originally from http://en.opasnet.org/w/HIA
# Original R code described in more detail at https://github.com/jtuomist/ghg-notebooks/wiki/Health-impact-assessment

import pandas as pd
import numpy as np
import json
import urllib.request
import math
from copy import deepcopy
from typing import Any, Dict, Optional, Union


In [887]:
class Ovariable:
    # content is the dataframe with the estimates
    content: Optional[pd.DataFrame]
    
    # quantity: what the ovariable measures, e.g. exposure, exposure_response, disease_burden
    quantity: Optional[str]
    
    def __init__(self, quantity: Optional[str] = None, content: Optional[pd.DataFrame] = None,
                 name: Optional[str] = None, input_nodes: Optional[list] = None,
                 meta: Optional[list] = None, unit: Optional[str] = None):
        self.name = name
        if quantity is not None:
            self.quantity = quantity # if quantity is not None else self.quantity
        self.content = content
        self.meta = meta
        self.unit = unit
        self.input_nodes = input_nodes
            
    def merge(self, other):
        
        def add_temporary_index(self):
            tst = self.index.to_frame().assign(temporary=1)
            tst = pd.MultiIndex.from_frame(tst)
            return self.set_index(tst)

        if isinstance(other, Ovariable):
            df2 = other.content
        else:
            df2 = pd.DataFrame([other],columns = ["Result"])
            
        df1 = add_temporary_index(self.content)
        df2 = add_temporary_index(df2)
        
        out = df1.merge(df2, left_index = True, right_index = True)
        out.index = out.index.droplevel(['temporary'])
        
        return Ovariable('', out)
    
    def clean(self):
        self.content = self.content.drop(['Result_x','Result_y'], axis='columns') 
        self.content = self.content.copy()
        return self

    def __add__(self, other):
        out = self.merge(other)
        out.content['Result'] = out.content['Result_x'] + out.content['Result_y']
        return out.clean()
    
    def __sub__(self, other):
        out = self.merge(other)
        out.content['Result'] = out.content['Result_x'] - out.content['Result_y']
        return out.clean()    

    def __mul__(self, other):
        out = self.merge(other)
        out.content['Result'] = out.content['Result_x'] * out.content['Result_y']
        return out.clean()    

    def __truediv__(self, other):
        out = self.merge(other)
        out.content['Result'] = out.content['Result_x'] / out.content['Result_y']
        return out.clean()

    def __mod__(self, other):
        out = self.merge(other)
        out.content['Result'] = out.content['Result_x'] % out.content['Result_y']
        return out.clean()

    def __pow__(self, other):
        out = self.merge(other)
        out.content['Result'] = out.content['Result_x'] ** out.content['Result_y']
        return out.clean()

    def __floordiv__(self, other):
        out = self.merge(other)
        out.content['Result'] = out.content['Result_x'] // out.content['Result_y']
        return out.clean()

    def __lt__(self, other):
        out = self.merge(other)
        out.content['Result'] = out.content['Result_x'] < out.content['Result_y']
        return out.clean()

    def __le__(self, other):
        out = self.merge(other)
        out.content['Result'] = out.content['Result_x'] <= out.content['Result_y']
        return out.clean()

    def __gt__(self, other):
        out = self.merge(other)
        out.content['Result'] = out.content['Result_x'] > out.content['Result_y']
        return out.clean()

    def __ge__(self, other):
        out = self.merge(other)
        out.content['Result'] = out.content['Result_x'] >= out.content['Result_y']
        return out.clean()

    def __eq__(self, other):
        out = self.merge(other)
        out.content['Result'] = out.content['Result_x'] == out.content['Result_y']
        return out.clean()

    def __ne__(self, other):
        out = self.merge(other)
        out.content['Result'] = out.content['Result_x'] != out.content['Result_y']
        return out.clean()

    def log(self):
        out =  np.log(self.content)
        return Ovariable(quantity = self.quantity, content = out, name = self.name,
                         meta = self.meta, unit = self.unit)

    def exp(self):
        out =  np.exp(self.content)
        return Ovariable(quantity = self.quantity, content = out, name = self.name,
                         meta = self.meta, unit = self.unit)    

#    def loc(self, condition):
#        self.content = self.content.loc[condition]
#        return self

In [888]:
# Exposure is the intensity of contact with the environment by the target population.

class Exposure(Ovariable):
    
    quantity = 'exposure'

    def compute(self):
        input_nodes = self.input_nodes

        for node in input_nodes:
            if node.quantity == 'consumption':
                consumption = node
            if node.quantity == 'concentration':
                concentration = node

        exposure = consumption * concentration

        return Exposure(content = exposure.content, input_nodes = self.input_nodes,
                       name = self.name, meta = self.meta, unit = self.unit)

In [1047]:
# Consumption is the amount of medium (food, water, air) consumed per time unit.

consumption = pd.DataFrame([['TEQ','child',20],['Fluoride','adult',4], ['PM2.5','adult',3],
                           ['campylobacter','child',5],['norovirus','adult',6],
                            ['giardia','child',7], ['Omega3','adult',8]],
                           columns=['exposure_agent','Age','Result']).set_index(['exposure_agent','Age'])
consumption = Ovariable(content = consumption, quantity='consumption')

# Concentration is the concentration of the exposure agent in the medium.

concentration = Ovariable(content = pd.DataFrame([['adult',2.5],['child',1.5]],columns=['Age','Result']).set_index(['Age']),
              quantity='concentration')
expo = Exposure(input_nodes=[consumption,concentration], name='pm_exposure', unit='ug/m3').compute()
expo.content

Result
Age   exposure_agent        
adult Fluoride          10.0
      PM2.5              7.5
      norovirus         15.0
      Omega3            20.0
child TEQ               30.0
      campylobacter      7.5
      giardia           10.5

In [1048]:
# bw is the body weight

bw = Ovariable('body_weight',
    content = pd.DataFrame({
        'Age':['child','adult'],
        'Result':[15,75]
    }).set_index('Age'))
bw.content

,Result
Age,
child,15
adult,75


In [1049]:
# Dose is the exposure scaled by logarithmic function or body weight

class Dose(Ovariable): # FIXIT: make this a function inside RR and AI instead
    quantity = 'dose'
    pass

    def compute(self):
        for node in self.input_nodes:
            if node.quantity == 'exposure':
                exposure = node
            if node.quantity == 'body_weight':
                bw = node

        outlog = np.log10(exposure.content).assign(scaling='Log10')

        outbw = exposure / bw 
        outbw = outbw.content.assign(scaling='BW')

        out = exposure.content.assign(scaling='None').append(outlog).append(outbw)
        ind = out.index.to_frame()
        ind = ind.assign(scaling=out['scaling'])
        ind = pd.MultiIndex.from_frame(ind)
        out = out.set_index(ind).drop(['scaling'], axis='columns')
        out = Dose(quantity = self.quantity, content = out, name=self.name,
                  meta = self.meta, unit = self.unit, input_nodes=self.input_nodes)

        return out

dose = Dose('dose', input_nodes=[expo, bw], name='pm_dose').compute()

expo.content

Result
Age   exposure_agent        
adult Fluoride          10.0
      PM2.5              7.5
      norovirus         15.0
      Omega3            20.0
child TEQ               30.0
      campylobacter      7.5
      giardia           10.5

In [1112]:
# Exposure-response function (ERF) is a variable that typically comes from data.
# Data comes from Opasnet [[ERFs of environmental pollutants]]
# http://en.opasnet.org/w/Special:Opasnet_Base?id=op_en5827
# The ovariables are converted to exposure_response_functions.csv by using code
# https://github.com/jtuomist/watch_network/create_erf_csv.R

class Erf(Ovariable):
    quantity = 'ERF'
    pass

#df = df.loc(lambda x: x['exposure_agent']=='PM2.5')
#df['exposure_agent']=='PM2.5'

df = pd.read_csv('exposure_response_functions.csv').drop(['source','hepatitis','age','exposure'], axis=1)

tmp = ['er_function','observation','scaling','exposure_agent','response']
tmp = tmp + list(set(df.columns) - set(tmp) - {'Result'})

df = df[tmp + ['Result']].replace({
    'ERS':'UR', # Just one er_function name per equation
    'CSF':'UR',
    'OR':'RR', # treat odds ratio as risk ratio although it is close only at small risk levels
    'TWI':'Step',
    'TDI':'Step',
    'ADI':'Step',
    'RDI':'Step',
    'NOAEL':'Step'
})
erf = df.sort_values(tmp).set_index(tmp)
erf = Erf(content=erf, name='pm_erf')    

erf.content

Result
er_function        observation scaling exposure_agent          response                 exposure_unit          
RR                 ERF         None    ALA                     CHD2 mortality           mg /day        0.999949
                                       Arsenic                 Bladder cancer morbidity µg /l          1.002000
                                       Chlorination byproducts Bladder cancer morbidity netrev /l      1.000029
                                                                                        µg /l          1.003900
                                       Dampness damage         Asthma morbidity         %              1.370000
...                                                                                                         ...
exact beta poisson Threshold   None    norovirus               norovirus infection      ?              0.055000
                                       rotavirus               rotavirus infection      ?              0.191000
                                       sapovirus               sapovirus infection      ?              0.055000
exponential        ERF         None    giardia                 giardia infection        ?              0.019900
                   Threshold   None    giardia                 giardia infection        ?              0.000000

[128 rows x 1 columns]

In [1111]:
# Frexposed is the fraction of exposed individuals within the target population. Defaults to 1
# but may be indexed by population subgroups.

frexposed = pd.DataFrame({
        'Age':['child','adult'],
        'Result':[1,1]
    }).set_index('Age')
frexposed = Ovariable(quantity = 'frexposed', content = frexposed, name='pm_frexposed')

frexposed.content

,Result
Age,
child,1
adult,1


In [1110]:
# P_illness is the probability of illness. Relevant for microbial infection endpoints.
# Typically a microbe-specific constant.

p_illness = pd.DataFrame({
        'response':['campylobacter infection','giardia infection','norovirus infection'],
        'Result':[1,1,1]
    }).set_index('response')
p_illness = Ovariable(quantity = 'p_illness', content = p_illness, name = 'p_illness_microbe')
p_illness.content

,Result
response,
campylobacter infection,1
giardia infection,1
norovirus infection,1


In [1053]:
# Relative risk (RR) is the risk of an exposed individual compared with a counterfactual
# unexposed individual using the modelled exposures. 

class Rr(Ovariable):
    quantity = 'RR'
    pass

    def compute(self):
        for node in self.input_nodes:
            if node.quantity == 'ERF':
                erf = node
            if node.quantity == 'dose':
                dose = node

        rr = Ovariable('temp',content = erf.content.loc[('RR','ERF')]) 
        threshold = Ovariable('temp',content = erf.content.loc[('RR','Threshold')])
        dose2 = (dose - threshold)#.dropna()
        dose2.content = np.clip(dose2.content, 0, None) # Smallest allowed value is 0

        t1 = (rr.log() * dose2).exp() #.dropna()

        Imax = Ovariable(content = erf.content.loc[('Relative Hill','ERF')])
        ed50 = Ovariable(content = erf.content.loc[('Relative Hill','Threshold')])

        t2 = ((dose2 * Imax))# / (dose + ed50) + 1)#.dropna()

        out = t1.content.append(t2.content)

        return Ovariable(quantity = 'RR', content = out, name = self.name, meta = self.meta, unit = 'unit')

rr = Rr(name='rr', input_nodes = [erf, dose]).compute()
erf.content

Result
er_function        observation scaling exposure_agent          response                 exposure_unit          
RR                 ERF         None    ALA                     CHD2 mortality           mg /day        0.999949
                                       Arsenic                 Bladder cancer morbidity µg /l          1.002000
                                       Chlorination byproducts Bladder cancer morbidity netrev /l      1.000029
                                                                                        µg /l          1.003900
                                       Dampness damage         Asthma morbidity         %              1.370000
...                                                                                                         ...
exact beta poisson Threshold   None    norovirus               norovirus infection      ?              0.055000
                                       rotavirus               rotavirus infection      ?              0.191000
                                       sapovirus               sapovirus infection      ?              0.055000
exponential        ERF         None    giardia                 giardia infection        ?              0.019900
                   Threshold   None    giardia                 giardia infection        ?              0.000000

[128 rows x 1 columns]

In [1054]:
incidence = pd.DataFrame({
        'response':['Liver cancer','Fluorosis','MeHg TWI','campylobacter infection',
                    'norovirus infection','giardia infection',"Loss in child's IQ points"],
        'Result':[1,1,1,1,1,1,1]
    }).set_index(['response'])
incidence = Ovariable('incidence', content = frexposed.content)
incidence.content

,Result
Age,
child,1
adult,1


In [1124]:
## Population attributable fraction PAF

#def paf(

class Paf(Ovariable):
    quantity = 'PAF'
    
    def compute(self):
        for node in self.input_nodes:
            if node.quantity == 'ERF':
                erf = node
            if node.quantity == 'dose':
                dose = node
            if node.quantity == 'frexposed':
                frexposed = node
            if node.quantity == 'incidence':
                incidence == node
            if node.quantity == 'RR':
                rr = node
            if node.quantity == 'p_illness':
                p_illness == node

        funcs = erf * dose
        funcs = list(set(funcs.content.reset_index().er_function))

        out = pd.DataFrame()

        for func in funcs:
            param1 = copy.deepcopy(erf)
            param1.content = param1.content.loc[(func,'ERF')]
            param2 = copy.deepcopy(erf)
            param2.content = param2.content.loc[(func,'Threshold')]

            if func == 'UR':
                k = param1
                threshold = param2
                dose2 = (dose - threshold)#.dropna()
                dose2.content = np.clip(dose2.content, 0, None) # Smallest allowed value is 0
                out1 = (k * dose2 * frexposed / incidence)#.dropna()
                out = out.append(out1.content.reset_index())

            if func == 'Step':
                upper = param1
                lower = param2
                out2 = (dose >= lower) * (dose <= upper) * -1 + 1
                out2 = out2 * frexposed / incidence
                out = out.append(out2.content.reset_index())

            if func == 'RR' or func == 'Relative Hill':
                r = frexposed * (rr - 1)
                out3 = (r > 0) * (r/(r + 1)) + (r <= 0) * r
                out = out.append(out3.content.reset_index())

            if func == 'beta poisson approximation':
                out4 = ((dose/param2 + 1)**(param1 * -1) * -1 + 1) * frexposed
                out4 = (out4 / incidence * p_illness)#.dropna() # dropna is needed before an index with NaN is used for merging
                out = out.append(out4.content.reset_index())

            if func == 'exact beta poisson':
                out5 = (exp(param1/(param1 + param2) * p2 * -1) * -1 + 1) * frexposed
                out5 = out5 / incidence * p_illness
                out = out.append(out5.content.reset_index())

            if func == 'exponential':
                k = param1
                out6 = (exp(k * dose * -1) * -1 + 1) * frexposed
                out6 = out6 / incidence * p_illness
                out = out.append(out6.content.reset_index())

        #keep = set(out.columns[out.notna().any()]) # remove indices that are empty
        #fill = set(out.columns[out.isna().any()]) # fill indices that have some empty locations
        #out = fillna(out, list(fill.intersection(keep) - {'result'}))

        keep = set(out.columns)- {'scaling','matrix','exposure','exposure_unit',0}
        out = out[list(keep)].set_index(list(keep - {'Result'}))

        return Paf(content = out, name = self.name, meta = self.meta, unit = self.unit)

paf = Paf(name = 'paf', input_nodes = [erf, dose, incidence, frexposed, rr, p_illness])
paf = paf.compute()
paf.content

Result
exposure_agent Age   response                                                    
TEQ            child Dioxin recommendation tolerable daily intake        0.000000
                     Dioxin recommendation tolerable daily intake 2018   1.000000
Fluoride       adult Fluorosis                                           1.250000
                     Fluorosis                                           1.250000
                     Fluorosis                                           0.000000
                     Fluorosis                                           0.000000
TEQ            child Cancer morbidity                                    0.001000
                     Yes or no developmental dental defects incl. ag...  0.384052
                     Yes or no tooth defect                              0.088627
                     Sperm concentration                                 0.001800
                     Yes or no dental defect                             0.041729
Omega3         adult Breast cancer                                      -0.010206
                     CHD2 mortality                                     -0.001451
                     CHD3 mortality                                     -0.009755
                     Coronary heart disease mortality                   -0.039249
                     Stroke mortality                                   -0.003992
PM2.5          adult Cardiopulmonary mortality                           0.064990
                     Lung cancer mortality                               0.099020
                     Total mortality                                     0.045298
None           adult CHD2 mortality                                     -4.400000
                     Stroke mortality                                   -3.400000
norovirus      adult norovirus infection                                 0.998192
campylobacter  child campylobacter infection                             0.144979
Omega3         adult Breast cancer                                      -0.010206
                     CHD2 mortality                                     -0.001451
                     CHD3 mortality                                     -0.009755
                     Coronary heart disease mortality                   -0.039249
                     Stroke mortality                                   -0.003992
PM2.5          adult Cardiopulmonary mortality                           0.064990
                     Lung cancer mortality                               0.099020
                     Total mortality                                     0.045298
None           adult CHD2 mortality                                     -4.400000
                     Stroke mortality                                   -3.400000
giardia        child giardia infection                                   0.188564

In [1125]:
# Population is typically indexed by subgroups.

population = Ovariable('population', content = pd.DataFrame({
        'Age':['adult','adult','child','child'],
        'sex':['male','female','male','female'],
        'Result':[100000]*4
    }).set_index(['Age','sex']))

population.content

Result
Age   sex           
adult male    100000
      female  100000
child male    100000
      female  100000

In [1126]:
# Case_burden is the disease burden of a single case of disease. This may be indexed by population subgroup e.g. age.

case_burden = Ovariable(quantity = 'case_burden', content = pd.DataFrame({
        'response':['Fluorosis', 'Liver cancer', "Loss in child's IQ points",
       'MeHg TWI', 'Breast cancer', 'CHD2 mortality', 'CHD3 mortality',
       'Coronary heart disease mortality', 'Stroke mortality',
       'Cardiopulmonary mortality', 'Lung cancer mortality',
       'Total mortality', 'CHD arrythmia mortality',
       'campylobacter infection', 'norovirus infection',
       'giardia infection'],
        'Result':[3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3]
    }).set_index(['response']))

case_burden.content

,Result
response,
Fluorosis,3
Liver cancer,3
Loss in child's IQ points,3
MeHg TWI,3
Breast cancer,3
CHD2 mortality,3
CHD3 mortality,3
Coronary heart disease mortality,3
Stroke mortality,3


In [1127]:
# BoD is the current (observed) burden of disease (measured in disability-adjusted life years or DALYs).

class Bod(Ovariable):
    quantity = 'disease_burden'
    
    def compute(self):
        for node in self.input_nodes:
            if node.quantity == 'incidence':
                incidence = node
            if node.quantity == 'population':
                population == node
            if node.quantity == 'case_burden':
                case_burden == node
                
        out = incidence * population * case_burden

        return Bod(content = out.content, name = self.name, meta = self.meta, unit = self.unit)

bod = Bod(input_nodes = [incidence,population,case_burden]).compute()
bod.content

Result
Age   sex    response                         
adult male   Fluorosis                  300000
             Liver cancer               300000
             Loss in child's IQ points  300000
             MeHg TWI                   300000
             Breast cancer              300000
...                                        ...
child female Total mortality            300000
             CHD arrythmia mortality    300000
             campylobacter infection    300000
             norovirus infection        300000
             giardia infection          300000

[64 rows x 1 columns]

In [1129]:
# bod_attr is the burden of disease that can be attributed to the exposure of interest.

class Bod_attr(Ovariable):
    quantity = 'bod_attr'
    
    def compute(self):
        for node in self.input_nodes:
            if node.quantity == 'disease_burden':
                bod = node
            if node.quantity == 'PAF':
                paf = node

        out = bod * paf
    
        return Bod_attr(content = out.content, name = self.name, meta = self.meta, unit = self.unit)
    
bod_attr = Bod_attr(input_nodes = [bod, paf], name = 'bod_attr').compute()
bod_attr.content

Result
Age   response                         sex    exposure_agent              
adult Breast cancer                    male   Omega3         -3.061857e+03
                                              Omega3         -3.061857e+03
                                       female Omega3         -3.061857e+03
                                              Omega3         -3.061857e+03
      CHD2 mortality                   male   Omega3         -4.352997e+02
                                              None           -1.320000e+06
                                              Omega3         -4.352997e+02
                                              None           -1.320000e+06
                                       female Omega3         -4.352997e+02
                                              None           -1.320000e+06
                                              Omega3         -4.352997e+02
                                              None           -1.320000e+06
      CHD3 mortality                   male   Omega3         -2.926354e+03
                                              Omega3         -2.926354e+03
                                       female Omega3         -2.926354e+03
                                              Omega3         -2.926354e+03
      Cardiopulmonary mortality        male   PM2.5           1.949700e+04
                                              PM2.5           1.949700e+04
                                       female PM2.5           1.949700e+04
                                              PM2.5           1.949700e+04
      Coronary heart disease mortality male   Omega3         -1.177471e+04
                                              Omega3         -1.177471e+04
                                       female Omega3         -1.177471e+04
                                              Omega3         -1.177471e+04
      Fluorosis                        male   Fluoride        3.750000e+05
                                              Fluoride        3.750000e+05
                                              Fluoride        0.000000e+00
                                              Fluoride        0.000000e+00
                                       female Fluoride        3.750000e+05
                                              Fluoride        3.750000e+05
                                              Fluoride        0.000000e+00
                                              Fluoride        0.000000e+00
      Lung cancer mortality            male   PM2.5           2.970588e+04
                                              PM2.5           2.970588e+04
                                       female PM2.5           2.970588e+04
                                              PM2.5           2.970588e+04
      Stroke mortality                 male   Omega3         -1.197723e+03
                                              None           -1.020000e+06
                                              Omega3         -1.197723e+03
                                              None           -1.020000e+06
                                       female Omega3         -1.197723e+03
                                              None           -1.020000e+06
                                              Omega3         -1.197723e+03
                                              None           -1.020000e+06
      Total mortality                  male   PM2.5           1.358952e+04
                                              PM2.5           1.358952e+04
                                       female PM2.5           1.358952e+04
                                              PM2.5           1.358952e+04
      norovirus infection              male   norovirus       2.994577e+05
                                       female norovirus       2.994577e+05
child campylobacter infection          male   campylobacter   4.349376e+04
                                       female campylobacter   4.349376e+04
      giardia infe

In [1122]:
#  sumExposcen subtracts the PAF of BAU scenario from the scenario of interest,
# thus giving the impact the exposure. The actual subtracting is done after the ovariable
# is evaluated by using CollapseMarginal (a standard function to manipulate ovariables in OpasnetUtils).

def sum_exposcen(out):
    if (pd.Series(['Exposcen']).isin(out.index.names).any()):
        out = out * pd.DataFrame({'Exposcen':['BAU','No exposure'], 'result':[1, -1]}).set_index('Exposcen')
        tmp = list(set(out.index.names) - {'Exposcen'})
        #ou = set(['Exposcen'])
        out = out.groupby(tmp).sum()
        
    return out

# fillna fills the NaN values of a column with all values available.
# object is pandas dataframe
# cols is a list of column names to fill

def fillna(object, cols):
    ind = object.index.names
    out = object.reset_index()
    if cols != [None]:
        for i in list(cols):
            a1 = out[out[i].notna()]
            a2 = out[out[i].isna()].drop(i, axis=1).assign(tmp=1)
            addition = pd.DataFrame({i:pd.unique(a1[i]), 'tmp':1})
            a2 = a2.merge(addition).drop('tmp',axis=1)
            out = a1.append(a2)
    if ind != [None]:
        out = out.set_index(ind)
    return out

In [ ]:
# mc2d function is not needed until we start using Monte Carlo and Iter in multi-index

"""
    mc2d is a function that samples the ovariable that describes individuals and then aggregates (typically averages over) the samples to reflect the situation of a defined population or population subgroups. This is done because in disease burden assessments, we are typically interested in population-level uncertainties rather than individual uncertainties.

    mc2d
    function (ova, mc2dpar = NULL)
    {
    if (is.null(mc2dpar))
    if (exists("mc2dparam"))
    mc2dpar <- mc2dparam
    else stop("Parameter list mc2dparam missing!\n")
    if (mc2dpar$run2d) {
    ova <- ova * mc2dpar$info
    require(reshape2)
    marg <- setdiff(c(colnames(ova@output)[ova@marginal],
    mc2dpar$newmarginals), "Iter")
    out <- aggregate(result(ova), by = ova@output[colnames(ova@output) %in%
    marg], FUN = function(x) {
    strength <- if (is.null(mc2dpar$strength))
    length(x)
    else mc2dpar$strength
    apply(array(as.numeric(sample(as.character(x), strength *
    mc2dpar$N2, replace = TRUE)), dim = c(strength,
    mc2dpar$N)), MARGIN = 2, FUN = mc2dpar$fun)
    })
    temp <- melt(out[[length(out)]])
    out[[length(out)]] <- 1:nrow(out)
    colnames(temp) <- c("Nrow", "Iter", "Result")
    out <- merge(out, temp, by.x = "x", by.y = "Nrow")
    out$x <- NULL
    out <- Ovariable(output = out, marginal = colnames(out) %in%
    c(marg, "Iter"))
    }
    else {
    out <- ova
    }
    return(out)
    }

"""
